In [31]:
import numpy as np
import pandas as pd
from datasets import load_dataset
import itertools

from torch import nn
from nltk import tokenize as nltk_tokenizer

dataset = load_dataset("McGill-NLP/feedbackQA")

Found cached dataset feedback_qa (/home/raja/.cache/huggingface/datasets/McGill-NLP___feedback_qa/plain_text/1.0.0/20c8f938f417c88303bb7041cea9554c1d14667686d7d7c5dda83dd4f39e5dc4)


  0%|          | 0/3 [00:00<?, ?it/s]

In [27]:
rating_scores = {'Excellent':3 , 'Acceptable':2 , 'Could be Improved':1, 'Bad': -1}

def process_df(df):
    df['question'] = df['question'].apply(lambda x: x.replace('\n',' '))
    df['answer'] = df['answer'].apply(lambda x: x.replace('\n',' '))
    df['list_feedback'] = df['feedback'].apply(lambda x: [ r + "___" + e for r,e in zip(x['rating'],x['explanation']) ])
    df['sampled_feedback'] = df['list_feedback'].apply(lambda x: np.random.choice(x).split("___") )
    df['rating_score'] = df['sampled_feedback'].apply(lambda x: rating_scores[x[0]])
    df['rating'] = df['sampled_feedback'].apply(lambda x: x[0])
    df['explanation'] = df['sampled_feedback'].apply(lambda x: x[1])
    return df

In [28]:
train_df = process_df(pd.DataFrame(dataset['train']))
val_df = process_df(pd.DataFrame(dataset['validation']))
test_df = process_df(pd.DataFrame(dataset['test']))

In [17]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader

# Load model from HuggingFace Hub
bert_chkpt = "sentence-transformers/all-mpnet-base-v2"
tokenizer = AutoTokenizer.from_pretrained(bert_chkpt)
model = AutoModel.from_pretrained(bert_chkpt)

In [18]:
tokenizer.all_special_tokens

['<s>', '</s>', '[UNK]', '<pad>', '<mask>']

In [19]:
train_df.head()

,question,answer,feedback,list_feedback,sampled_feedback,rating_score,rating,explanation
0,How do I get help finding a job?,Coronavirus (COVID-19) information for job see...,"{'rating': ['Excellent', 'Could be Improved'],...",[Excellent___Has a link to detailed informatio...,"[Could be Improved, This answer provides a lin...",1,Could be Improved,"This answer provides a link for job searches, ..."
1,How do I get help finding a job?,Coronavirus (COVID-19) information for job see...,"{'rating': ['Excellent', 'Excellent'], 'explan...",[Excellent___A link to a job search website is...,"[Excellent, A link to a job search website is ...",3,Excellent,"A link to a job search website is included, as..."
2,How do I get help finding a job?,Coronavirus (COVID-19) information and support...,"{'rating': ['Bad', 'Acceptable'], 'explanation...",[Bad___Talks about tax credits for businesses ...,"[Bad, Talks about tax credits for businesses t...",-1,Bad,Talks about tax credits for businesses that hi...
3,If I am in Australia on a worker holiday marke...,Frequently Asked Questions Working holiday mak...,"{'rating': ['Could be Improved', 'Acceptable']...",[Could be Improved___Answer is about Working H...,"[Acceptable, Answer is rather cut and dry but ...",2,Acceptable,Answer is rather cut and dry but is also a lit...
4,If I am in Australia on a worker holiday marke...,Frequently Asked Questions COVID-19 Pandemic -...,"{'rating': ['Bad', 'Could be Improved'], 'expl...",[Bad___Discusses pandemic visas. Doesn't menti...,"[Bad, Discusses pandemic visas. Doesn't mentio...",-1,Bad,Discusses pandemic visas. Doesn't mention the ...


In [20]:
train_df['answer'].loc[0]

'Coronavirus (COVID-19) information for job seekers Exisiting job seekers If you are a current job seeker or participant, this fact sheet provides important information about mutual obligation requirements, appointments with your provider, and what to do if you are self-isolating:  Information for job seekers and participants  If you are participating in the ParentsNext program, this fact sheet provides important information about your activities and appointments.   Information for ParentsNext participants   ParentsNext participants Frequently Asked Questions   If you are a New Business Assistance with NEIS participant, these Frequently Asked Questions (FAQ) provides information about accessing the Coronavirus Supplement and what support is available during this time:  New Business Assistance with NEIS participants - Frequently Asked Questions  If you are a New Business Assistance with NEIS provider, these Frequently Asked Questions (FAQ) provides information about supporting NEIS part

In [21]:
tokenizer('Hello, how are you doing?'+ f" {tokenizer.eos_token} " + "Hemlooooo",add_special_tokens=True,return_tensors='pt', return_length=1)

{'input_ids': tensor([[    0,  7596,  1014,  2133,  2028,  2021,  2729,  1033,     2, 19614,
          4139,  9545,  9545,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'length': tensor([14])}

In [32]:
len(nltk_tokenizer.sent_tokenize(train_df['answer'].loc[0]))

3

In [33]:
tok_inp = tokenizer(nltk_tokenizer.sent_tokenize(train_df['answer'].loc[0]),add_special_tokens=False,return_token_type_ids=True)#,max_length=200,padding='max_length')
tok_inp

{'input_ids': [[21891, 23354, 1010, 2526, 17262, 1015, 2543, 1011, 2596, 2009, 3109, 24075, 4658, 17421, 3440, 3109, 24075, 2069, 2021, 2028, 1041, 2787, 3109, 29448, 2034, 13184, 1014, 2027, 2759, 7127, 3644, 2594, 2596, 2059, 8207, 14991, 5922, 1014, 14655, 2011, 2119, 10806, 1014, 2002, 2058, 2004, 2083, 2069, 2021, 2028, 2973, 1015, 11167, 22252, 1028, 2596, 2009, 3109, 24075, 2002, 6822, 2069, 2021, 2028, 8023, 2003, 2000, 3012, 2642, 18417, 2569, 1014, 2027, 2759, 7127, 3644, 2594, 2596, 2059, 2119, 3454, 2002, 14655, 1016], [2596, 2009, 3012, 2642, 18417, 6822, 3012, 2642, 18417, 6822, 4707, 2360, 3984, 2069, 2021, 2028, 1041, 2051, 2453, 5379, 2011, 11269, 2487, 13184, 1014, 2126, 4707, 2360, 3984, 1010, 6908, 4164, 1011, 3644, 2596, 2059, 3233, 2079, 2000, 21891, 23354, 12452, 2002, 2058, 2494, 2007, 2804, 2080, 2027, 2055, 1028, 2051, 2453, 5379, 2011, 11269, 2487, 6822, 1015, 4707, 2360, 3984, 2069, 2021, 2028, 1041, 2051, 2453, 5379, 2011, 11269, 2487, 10806, 1014, 2126, 47

In [246]:
tokenizer('Hello, how are you?',return_special_tokens_mask=True,add_special_tokens=True, padding='max_length', max_length=20)['special_tokens_mask']

[1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [43]:
import tqdm

class feedback_QA_dataset(Dataset):
    
    def __init__(self,df,max_length=512):
        self.df = df
        self.max_len = max_length
        self.data = []
        skipped = 0
        
        for i in tqdm.tqdm(range(len(self.df)),desc='vectorizing..'):
            
            d = {}
            
            tok_question = tokenizer('Question: ' + self.df.iloc[i]['question'] + ' Answer: ', add_special_tokens=False)
            tok_answer = tokenizer(self.df.iloc[i]['answer'], add_special_tokens=False)
            tok_feedback = tokenizer(self.df.iloc[i]['explanation'], add_special_tokens=False)
            
            if len(tok_question['input_ids']+tok_answer['input_ids']+tok_feedback['input_ids'])+4 > self.max_len:
                skipped +=1
                continue
            
            context = [tokenizer.bos_token_id] + tok_question['input_ids'] + tok_answer['input_ids']
            context_attn = [1] + tok_question['attention_mask'] + tok_answer['attention_mask']
            context_pool_mask = [0] + [0]*len(tok_question['input_ids']) + tok_answer['attention_mask']
            
            
            d['context_w_feedback'] = context + [tokenizer.sep_token_id]*2 + tok_feedback['input_ids'] + [tokenizer.eos_token_id]
            
            PAD_LEN = self.max_len - len(d['context_w_feedback'])
            
            d['context_w_feedback'] += [tokenizer.pad_token_id]*PAD_LEN
            d['context_w_feedback_attn'] = context_attn + [1,1] + tok_feedback['attention_mask'] + [1] + [0]*PAD_LEN            
            d['context'] = d['context_w_feedback']
            d['context_attn'] = context_attn + [1,0] + [0]*len(tok_feedback['attention_mask']) + [0] + [0]*PAD_LEN
            
            d['feedback_pool_mask'] = [0]*len(context_pool_mask) + [0,0] + tok_feedback['attention_mask'] + [0] + [0]*PAD_LEN
            d['answer_pool_mask'] = context_pool_mask + [0,0] + [0]*len(tok_feedback['attention_mask']) + [0] + [0]*PAD_LEN
            
            answer_phrases = nltk_tokenizer.sent_tokenize(self.df.iloc[i]['answer'])
            tok_phrases = tokenizer(answer_phrases,add_special_tokens=False,return_token_type_ids=True)
            
            d['answer_phrases_pool_mask'] = []
            
            for j in range(len(answer_phrases)):
                answer_phrases_attn_mask = tok_phrases['token_type_ids'].copy()
                answer_phrases_attn_mask[j] = tok_phrases['attention_mask'][j].copy()
                answer_phrases_attn_mask = list(itertools.chain.from_iterable(answer_phrases_attn_mask))
                # pad_len = len(tok_answer['attention_mask']) - len(answer_phrases_attn_mask)
                # answer_phrases_attn_mask += [0]*pad_len
                
                answer_phrase_pool_mask = [0] + [0]*len(tok_question['input_ids']) + answer_phrases_attn_mask + [0,0] + [0]*len(tok_feedback['attention_mask']) + [0] + [0]*PAD_LEN
                
                d['answer_phrases_pool_mask'].append(answer_phrase_pool_mask)
            
            if len(d['answer_phrases_pool_mask'][0])>len(d['answer_pool_mask']):
                skipped +=1
                continue
                
            self.data.append(d)
        print('Skipped: ',skipped)

    def add_neg_samples(self):
        for i in tqdm.tqdm(range(self.__len__()),desc='adding neg samples...'):
            self.data[i]['feedback_set'] = [self.data[i]['context_w_feedback']]
            self.data[i]['feedback_attn_set'] = [self.data[i]['context_w_feedback_attn']]
            self.data[i]['feedback_pool_mask_set'] = [self.data[i]['feedback_pool_mask']]
            L = list(range(self.__len__()))
            L.remove(i)
            neg_samples_idx = np.random.choice(L,size=4)
            for n_id in neg_samples_idx:
                self.data[i]['feedback_set'].append(self.data[n_id]['context_w_feedback'])
                self.data[i]['feedback_attn_set'].append(self.data[n_id]['context_w_feedback_attn'])
                self.data[i]['feedback_pool_mask_set'].append(self.data[n_id]['feedback_pool_mask'])
            for k in self.data[i].keys():
                self.data[i][k] = torch.tensor(self.data[i][k])
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        return self.data[idx]

In [44]:
train_dataset = feedback_QA_dataset(train_df)
train_dataset.add_neg_samples()
valid_dataset = feedback_QA_dataset(val_df)
valid_dataset.add_neg_samples()
test_dataset = feedback_QA_dataset(test_df)
test_dataset.add_neg_samples()

vectorizing..: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5660/5660 [00:17<00:00, 315.29it/s]


Skipped:  436


vectorizing..: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1410/1410 [00:04<00:00, 325.30it/s]


Skipped:  105


vectorizing..: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1995/1995 [00:06<00:00, 317.23it/s]


Skipped:  174


adding neg samples...: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1821/1821 [00:07<00:00, 253.42it/s]


In [45]:
train_DL = DataLoader(train_dataset,batch_size=1,shuffle=True)
valid_DL = DataLoader(valid_dataset,batch_size=1,shuffle=True)
test_DL = DataLoader(test_dataset,batch_size=1,shuffle=False)

In [46]:
for b in train_DL:
    for k in b.keys():
        print(k,b[k].shape)
    break

context_w_feedback torch.Size([1, 512])
context_w_feedback_attn torch.Size([1, 512])
context torch.Size([1, 512])
context_attn torch.Size([1, 512])
feedback_pool_mask torch.Size([1, 512])
answer_pool_mask torch.Size([1, 512])
answer_phrases_pool_mask torch.Size([1, 11, 512])
feedback_set torch.Size([1, 5, 512])
feedback_attn_set torch.Size([1, 5, 512])
feedback_pool_mask_set torch.Size([1, 5, 512])


In [78]:
t = torch.tensor([[[1,2,3],[4,5,6]]])
t.shape,t.repeat(2,1,1).shape

(torch.Size([1, 2, 3]), torch.Size([2, 2, 3]))

In [81]:
from transformers import BartForConditionalGeneration

device = 'cuda:0'

model = AutoModel.from_pretrained(bert_chkpt).to(device)

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    se = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return F.normalize(se, p=2, dim=1)

j = 0

with torch.no_grad():
    for b in test_DL:
        se = mean_pooling( model(input_ids = b['context_w_feedback'].to(device),attention_mask=b['context_attn'].to(device))[0], b['answer_pool_mask'].to(device))
        
        fmo = model(input_ids = b['feedback_set'][0].to(device),attention_mask=b['feedback_attn_set'][0].to(device))[0]
        # print(b['feedback_set'][0].shape, b['feedback_attn_set'][0].shape, fmo.shape, b['feedback_pool_mask_set'][0].shape)
        
        fe = mean_pooling(model(input_ids = b['feedback_set'][0].to(device),attention_mask=b['feedback_attn_set'][0].to(device))[0], b['feedback_pool_mask_set'][0].to(device))
        
        pmo = model(input_ids = b['context_w_feedback'].to(device),attention_mask=b['context_attn'].to(device))
        # print(pmo[0].shape,b['answer_phrases_pool_mask'].shape,pmo[0].repeat(b['answer_phrases_pool_mask'][0].shape[0],1,1).shape,b['answer_phrases_pool_mask'][0].shape)
        pe = mean_pooling(pmo[0].repeat(b['answer_phrases_pool_mask'][0].shape[0],1,1),b['answer_phrases_pool_mask'][0].to(device) )# for i in range(b['answer_phrases_pool_mask'][0].shape[0])]
        # pe = torch.stack(pe).squeeze(1)
        cos_sim = F.cosine_similarity(se,fe,dim=1)
        cos_phrase_sim = torch.matmul(pe,fe.transpose(1,0))
        print(fe.shape,se.shape,pe.shape,cos_sim,cos_phrase_sim.mean(0))
        
        sent_probs = F.softmax(cos_sim,dim=-1)
        phrase_probs = F.softmax(cos_phrase_sim,dim=-1)
        
        # print('\nInput: ',tokenizer.decode(torch.mul(b['context_w_feedback'][0],b['context_attn'][0]),skip_special_tokens=True),'\n')
        # print('Feedback: ',tokenizer.decode(torch.mul(b['context_w_feedback'][0],b['feedback_pool_mask'][0]),skip_special_tokens=True),'\n')
#         for i in range(b['answer_phrases_pool_mask'][0].shape[0]):
#             relevance = phrase_probs[i][0] - sent_probs[0]
            
#             phrase_tok = torch.mul(b['context_w_feedback'][0],b['answer_phrases_pool_mask'][0][i])
#             print(f"Phrase {i}:",tokenizer.decode(phrase_tok,skip_special_tokens=True))
#             print(f"Relevance of phrase {i} is {relevance}",'\n')
        
        print('softmax: ',F.softmax(cos_sim),F.softmax(cos_phrase_sim,dim=-1))
        
        tgt_tensor = torch.zeros(b['feedback_set'].shape[1] , device=device)
        tgt_tensor[0] = 1.0
        print('CE Loss: ', F.cross_entropy(cos_sim,target=tgt_tensor), F.cross_entropy(cos_phrase_sim.mean(0),target=torch.tensor([1.0,0,0,0,0]).to(device)))
        print('----------------------------')
        j+=1
        if j>5:
            break

del model

torch.Size([5, 768]) torch.Size([1, 768]) torch.Size([2, 768]) tensor([0.6895, 0.3283, 0.2529, 0.2336, 0.3751], device='cuda:0') tensor([0.6700, 0.3212, 0.2468, 0.2296, 0.3667], device='cuda:0')
softmax:  tensor([0.2698, 0.1880, 0.1743, 0.1710, 0.1970], device='cuda:0') tensor([[0.2579, 0.1916, 0.1736, 0.1766, 0.2003],
        [0.2766, 0.1854, 0.1763, 0.1675, 0.1942]], device='cuda:0')
CE Loss:  tensor(1.3102, device='cuda:0') tensor(1.3198, device='cuda:0')
----------------------------


/home/raja/Ravi/rghadia_env/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([5, 768]) torch.Size([1, 768]) torch.Size([2, 768]) tensor([0.6603, 0.2329, 0.2974, 0.2593, 0.1634], device='cuda:0') tensor([0.6352, 0.2273, 0.2901, 0.2499, 0.1616], device='cuda:0')
softmax:  tensor([0.2758, 0.1799, 0.1919, 0.1847, 0.1678], device='cuda:0') tensor([[0.2620, 0.1842, 0.1982, 0.1800, 0.1756],
        [0.2822, 0.1775, 0.1870, 0.1901, 0.1632]], device='cuda:0')
CE Loss:  tensor(1.2881, device='cuda:0') tensor(1.3018, device='cuda:0')
----------------------------
torch.Size([5, 768]) torch.Size([1, 768]) torch.Size([6, 768]) tensor([0.8366, 0.6493, 0.8126, 0.2180, 0.1444], device='cuda:0') tensor([0.8052, 0.6229, 0.7825, 0.2107, 0.1363], device='cuda:0')
softmax:  tensor([0.2601, 0.2157, 0.2539, 0.1401, 0.1302], device='cuda:0') tensor([[0.2623, 0.2151, 0.2516, 0.1408, 0.1302],
        [0.2608, 0.2125, 0.2557, 0.1447, 0.1263],
        [0.2564, 0.2148, 0.2568, 0.1435, 0.1285],
        [0.2532, 0.2189, 0.2490, 0.1431, 0.1357],
        [0.2568, 0.2137, 0.2518, 0.14

In [230]:
t = torch.tensor([[[1,2,3,4,5],[6,7,8,9,0]]])
t.repeat(2,1,1)

tensor([[[1, 2, 3, 4, 5],
         [6, 7, 8, 9, 0]],

        [[1, 2, 3, 4, 5],
         [6, 7, 8, 9, 0]]])

In [82]:
class discriminator(nn.Module):
    def __init__(self, model_chkpt, device='cuda:0'):
        super().__init__()
        
        self.model = AutoModel.from_pretrained(model_chkpt).to(device)
        self.device = device
        
    def mean_pooling(self,model_output,attention_mask):
        token_embeddings = model_output #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        se = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return F.normalize(se, p=2, dim=1)
        
    def forward(self, b):
        sent_model_out = self.model(input_ids = b['context_w_feedback'].to(self.device),attention_mask=b['context_attn'].to(self.device))[0]
        feedback_model_out = self.model(input_ids = b['feedback_set'][0].to(self.device),attention_mask=b['feedback_attn_set'][0].to(self.device))[0]
        
        sent_emb = self.mean_pooling( sent_model_out, b['answer_pool_mask'].to(self.device))
        feedback_emb = self.mean_pooling( feedback_model_out, b['feedback_pool_mask_set'][0].to(self.device))
        
        # print(pmo[0].shape,b['answer_phrases_pool_mask'].shape)
        phrase_emb = self.mean_pooling( sent_model_out.repeat(b['answer_phrases_pool_mask'][0].shape[0],1,1), b['answer_phrases_pool_mask'][0].to(self.device) )
        # phrase_emb = torch.stack(phrase_emb).squeeze(1)
        cos_sim = F.cosine_similarity(sent_emb,feedback_emb,dim=1)
        cos_phrase_sim = torch.matmul(phrase_emb,feedback_emb.transpose(1,0))
        
        tgt_tensor = torch.zeros(b['feedback_set'].shape[1] , device=self.device)
        tgt_tensor[0] = 1.0 #the relevant feedback is always present at index 0
        
        return_dict = {'sent_ce_loss': F.cross_entropy(cos_sim,target=tgt_tensor),
                       'avg_phrase_ce_loss': F.cross_entropy(cos_phrase_sim.mean(0),target=tgt_tensor),
                       'sent_probs': F.softmax(cos_sim,dim=-1),
                       'phrase_probs': F.softmax(cos_phrase_sim,dim=-1)}
        
        return return_dict
        
        

In [83]:
def train(discriminator,train_dl,valid_dl,epochs,batch_size,optimizer,PATIENCE=20,save_dir=None):

    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    
    discriminator.train()
    
    loss_acc = 0
    num_batches = 0
    total_steps = 0
    best_valid_loss = np.inf
    patience = PATIENCE
    
    train_loss_arr,valid_loss_arr = [],[]
    
    optimizer.zero_grad()
    discriminator.zero_grad()
    
    valid_loss = validate(discriminator,valid_dl)
    valid_loss_arr.append(valid_loss/len(valid_dl))
    
    for E in range(epochs):
        
        num_samples = 0
        
        for b in train_dl:
            
            y = discriminator(b)
                          # decoder_input_ids=b['feedback'].squeeze(1)[:,:-1].to(device),
                          # decoder_attention_mask=b['feedback_attn'].squeeze(1)[:,:-1].to(device))
            loss = y['sent_ce_loss'] + y['avg_phrase_ce_loss'] #F.cross_entropy(y.logits.permute(0,2,1), b['feedback'].squeeze(1)[:,1:].to(device), ignore_index=tokenizer.pad_token_id)
            
            num_samples+=1
            
            loss.backward()
            loss_acc += loss.item()
            
            if num_samples%batch_size==0:
                optimizer.step()

                num_batches += 1
                total_steps += 1
            
                train_loss_arr.append(loss_acc/num_batches)
                
                optimizer.zero_grad()
            
                if total_steps%100==0 and total_steps!=0:
                    print("Epoch:",E,"\t","Steps taken:",total_steps,"\tLoss:",loss_acc/num_batches)
            
        #print("Epoch:",E,"\t","Steps taken:",total_steps,"\tLoss:",loss_acc/num_batches)
        
        torch.save({'model_state':discriminator.state_dict(),
                    'optimizer':optimizer.state_dict(),
                    'epoch':E},
                    f"{save_dir}/Epoch_{E}_model_chkpt.pth.tar")
        
        valid_loss = validate(discriminator,valid_dl)
        valid_loss_arr.append(valid_loss/len(valid_dl))
        
        if valid_loss<best_valid_loss:
            best_valid_loss = valid_loss
            patience = PATIENCE
            
            torch.save({'model_state':discriminator.state_dict(),
                        'optimizer':optimizer.state_dict(),
                        'epoch':E},
                        f"{save_dir}/best_model_chkpt.pth.tar")
        else:
            patience -= 1
            print(f"REDUCING PATIENCE...{patience}")

        if patience<=0:
            print("RUNNING OUT OF PATIENCE... TERMINATING")
            break
    
    
    return train_loss_arr,valid_loss_arr
                

In [84]:
def validate(discriminator,valid_dl):
    
    discriminator.eval()
    valid_loss = 0
    with torch.no_grad():
        for b in valid_dl:
            y = discriminator(b)
                          # decoder_input_ids=b['feedback'].squeeze(1)[:,:-1].to(device),
                          # decoder_attention_mask=b['feedback_attn'].squeeze(1)[:,:-1].to(device))
            loss = y['sent_ce_loss'] + y['avg_phrase_ce_loss'] #F.cross_entropy(y.logits.permute(0,2,1), b['feedback'].squeeze(1)[:,1:].to(device), ignore_index=tokenizer.pad_token_id)
            valid_loss += loss.item()
            
    print("Validation Loss:",valid_loss)
    return valid_loss

In [86]:
import os

from transformers import AutoModel

EPOCHS = 50
BATCH_SIZE = 16

device = 'cuda:1'

# MPNet = AutoModel.from_pretrained(bert_chkpt).to(device)
discriminator_model = discriminator(bert_chkpt,device=device)

optimizer = torch.optim.AdamW(discriminator_model.parameters(),lr=1e-5)

save_dir = 'Detect_Span_FB_MPNET_chkpts_3'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

train_loss,valid_loss = train(discriminator_model,
                              train_DL,
                              valid_DL,
                              EPOCHS,
                              BATCH_SIZE,
                              optimizer,
                              PATIENCE=5,
                              save_dir=save_dir)

Validation Loss: 3223.12977540493
Epoch: 0 	 Steps taken: 100 	Loss: 29.962738287448882
Epoch: 0 	 Steps taken: 200 	Loss: 29.572392408251762
Epoch: 0 	 Steps taken: 300 	Loss: 29.432847184737522
Validation Loss: 2379.9055646657944
Epoch: 1 	 Steps taken: 400 	Loss: 29.339444583654405
Epoch: 1 	 Steps taken: 500 	Loss: 29.240687273740768
Epoch: 1 	 Steps taken: 600 	Loss: 29.17627572039763
Validation Loss: 2379.1063376665115
Epoch: 2 	 Steps taken: 700 	Loss: 29.125879089662007
Epoch: 2 	 Steps taken: 800 	Loss: 29.04649720802903
Epoch: 2 	 Steps taken: 900 	Loss: 28.987862569358615
Validation Loss: 2380.6113728284836
Epoch: 3 	 Steps taken: 1000 	Loss: 28.943698433041572
Epoch: 3 	 Steps taken: 1100 	Loss: 28.865703828659925
Epoch: 3 	 Steps taken: 1200 	Loss: 28.807999064425626
Epoch: 3 	 Steps taken: 1300 	Loss: 28.75654564353136
REDUCING PATIENCE...4
Validation Loss: 2387.3019028902054
Epoch: 4 	 Steps taken: 1400 	Loss: 28.691443885564805
Epoch: 4 	 Steps taken: 1500 	Loss: 28.625

In [ ]:
import json

with open('train_loss.json','w') as f:
    json.dump(train_loss,f)

with open('valid_loss.json','w') as f:
    json.dump(valid_loss,f)

In [ ]:
train_loss_ds = np.array(train_loss)[np.round(np.linspace(0, len(train_loss) - 1, len(valid_loss))).astype(int)]
loss_df = pd.DataFrame({'train_loss':train_loss_ds , 'valid_loss':valid_loss})

In [ ]:
from plotly import express as px
px.line(loss_df,y=['train_loss','valid_loss'])

In [ ]:
discriminator.load_state_dict(torch.load('GenFB_BART_chkpts_1/Epoch_0_model_chkpt.pth.tar')['model_state'])

In [ ]:
i = 0
for b in train_DL:
    out = discriminator.generate(inputs=b['input'][0:1,0].to(device),top_p=0.5)
    print(tokenizer.decode(b['input'][0:1,0][0],skip_special_tokens=True))
    print(tokenizer.decode(b['feedback'][0:1,0][0],skip_special_tokens=True))
    print(tokenizer.decode(out[0]))
    print("--------------------------------------------------------")
    i+=1
    if i>10:
        break